In [1]:
import os
import re
import openpyxl
import pickle
from PIL import Image, ImageDraw, ImageFont
import pandas as pd


DATA_DIR = "/home/ec2-user/word_level_ocr/pritom/datasets/"

### Get Wordlist from Dakshina

In [2]:
with open(os.path.join(DATA_DIR, "dakshina/wiki-filt.train.text.shuf.txt"), 'r') as wordfile:
# with open(os.path.join(DATA_DIR, "dakshina/wiki-filt.valid.text.shuf.txt"), 'r') as wordfile:
    lines = [line.rstrip() for line in wordfile]
    print("Total lines: {}".format(len(lines)))

Total lines: 895145


In [3]:
wordlist = []

for line in lines:
    words = line.split()
    for word in words:
        wordlist.append(word)
        
print(len(wordlist))
wordlist = list(set(wordlist))
print(len(wordlist))

10912210
717674


In [4]:
charlist = []

for word in wordlist:
    for c in word:
        charlist.append(c)
        
print(len(charlist))
charlist = list(set(charlist))
print(len(charlist))
print(charlist)

6236283
1892
['姫', '角', '雄', '¯', '紅', 'ꠋ', 'ʙ', 'ګ', 'ա', '冀', '\xad', '*', 'の', 'Ü', 'I', 'و', 'J', '시', 'ག', '\\', '客', '柱', 'ে', 'ط', '闕', '幹', 'с', 'া', '気', 'झ', '紫', 'ᱤ', '景', '準', 'ི', 'ὄ', '\u202a', '黒', '茅', '⁷', '林', '¹', '祖', '尼', 'ℓ', '播', 'ਾ', '?', '계', '৯', 'ን', 'ந', 'ジ', 'ح', '戦', 'Ὠ', 'ভ', '織', 'Ե', '玉', 'ୟ', 'E', 'ῶ', '孙', ':', 'ₐ', 'ไ', 'ग', '好', '微', '般', '8', 'డ', '支', 'ц', 'ヌ', 'ﺍ', '蹈', '拳', 'ଡ', '荷', '御', 'ব', '鸭', '撃', 'ਂ', 'ダ', '振', 'ο', 'ລ', '楊', 'ё', '米', 'ψ', '舊', 'г', '偵', '동', 'ᱠ', '見', '經', '有', '皿', 'ὁ', 'ٹ', 'ṛ', 'X', 'İ', 'າ', '祝', 'バ', '卦', 'ل', '₍', 'ر', "'", 'ꠑ', 'ヶ', 'う', 'ர', '鎮', '布', '皮', 'ζ', 'ਸ', '才', 'ے', 'ू', ',', 'ס', '呦', '坝', 'ဝ', 'ய', '―', 'ら', 'ན', 'ト', 'ູ', 'L', 'Ə', '다', 'ッ', '월', '庁', '枕', '賦', '=', '弁', '미', 'ಕ', '剎', 'ℏ', '宗', '型', 'ອ', 'ڑ', '9', 'е', '科', '็', '饂', '发', '方', '瀑', 'い', 'ゲ', 'ഹ', 'গ', 'ₘ', '📱', '論', '灣', '媛', 'ঊ', 'ܟ', 'ض', 'b', '行', 'ព', 'ꠞ', '杜', '川', '‡', 'ꠟ', '₿', 'ద', '于', 'ڈ', '౼', '♊', '場', 'ῦ', '港', '滋', 'ꠘ

In [5]:
ben_block = re.compile(u'[\u0980-\u09FF]+')
print(ben_block.fullmatch('এটাশুধুমাত্রসেনিটিচেকযেনরমালশব্দেঠিকআছেকিনা'))

<re.Match object; span=(0, 43), match='এটাশুধুমাত্রসেনিটিচেকযেনরমালশব্দেঠিকআছেকিনা'>


In [6]:
ben_wordlist = []
ben_block = re.compile(u'([\u0980-\u09FF])+')


for word in wordlist:
    if ben_block.fullmatch(word) != None:
        ben_wordlist.append(word)
    
print(len(ben_wordlist))

423697


In [7]:
def normalize_word(word):

    if 'ো' in word: word = word.replace('ো', 'ো')
    
    if 'ৗ' in word:    
        if 'ৌ' in word: word = word.replace('ৌ', 'ৌ') 
        else: word = word.replace('ৗ', 'ী') # 'ৗ' without 'ে' is replaced by 'ী'
    
    if '়' in word:
        if 'ব়' in word: word = word.replace('ব়', 'র')
        if 'য়' in word: word = word.replace('য়', 'য়')
        if 'ড়' in word: word = word.replace('ড়', 'ড়')
        if 'ঢ়' in word: word = word.replace('ঢ়', 'ঢ়')
        if '়' in word: word = word.replace('়', '') # discard any other '়' without 'ব'/'য'/'ড'/'ঢ'
        
    # visually similar '৷' (Bengali Currency Numerator Four) is replaced by '।' (Devanagari Danda)
    if '৷' in word: word = word.replace('৷', '।')
    
    return word
    
########################Version 4#################################################################
def extract_graphemes(word):
    
    forms_cluster = {'ক': ['ক', 'ট', 'ত', 'ন', 'ব', 'ম', 'র', 'ল', 'ষ', 'স'],
                     'গ': ['গ', 'ধ', 'ন', 'ব', 'ম', 'ল'],
                     'ঘ': ['ন'],
                     'ঙ': ['ক', 'খ', 'গ', 'ঘ', 'ম'],
                     'চ': ['চ', 'ছ', 'ঞ'],
                     'জ': ['জ', 'ঝ', 'ঞ', 'ব'],
                     'ঞ': ['চ', 'ছ', 'জ', 'ঝ'],
                     'ট': ['ট', 'ব'],
                     'ড': ['ড'],
                     'ণ': ['ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ব', 'ম'],
                     'ত': ['ত', 'থ', 'ন', 'ব', 'ম', 'র'],
                     'থ': ['ব'],
                     'দ': ['গ', 'ঘ', 'দ', 'ধ', 'ব', 'ভ', 'ম'],
                     'ধ': ['ন', 'ব'],
                     'ন': ['জ', 'ট', 'ঠ', 'ড', 'ত', 'থ', 'দ', 'ধ', 'ন', 'ব', 'ম', 'স'],
                     'প': ['ট', 'ত', 'ন', 'প', 'ল', 'স'],
                     'ফ': ['ট', 'ল'],
                     'ব': ['জ', 'দ', 'ধ', 'ব', 'ভ', 'ল'],
                     'ভ': ['র'],
                     'ম': ['ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'ল'],
                     'ল': ['ক', 'গ', 'ট', 'ড', 'প', 'ফ', 'ব', 'ম', 'ল', 'স'],
                     'শ': ['চ', 'ছ', 'ত', 'ন', 'ব', 'ম', 'ল'],
                     'ষ': ['ক', 'ট', 'ঠ', 'ণ', 'প', 'ফ', 'ব', 'ম'],
                     'স': ['ক', 'খ', 'ট', 'ত', 'থ', 'ন', 'প', 'ফ', 'ব', 'ম', 'ল'],
                     'হ': ['ণ', 'ন', 'ব', 'ম', 'ল'],
                     'ড়': ['গ']}
    
    forms_tripple_cluster = {'ক্ষ': ['ণ', 'ম'], 'ঙ্ক': ['ষ'], 'চ্ছ': ['ব'], 'জ্জ': ['ব'],
                             'ত্ত': ['ব'], 'দ্দ': ['ব'], 'দ্ধ': ['ব'], 'দ্ভ': ['র'],
                             'ন্ত': ['ব'], 'ন্দ': ['ব'], 'ম্প': ['ল'], 'ম্ভ': ['র'],
                             'ষ্ক': ['র'], 'স্ক': ['র'], 'স্ত': ['ব', 'র'], 'স্প': ['ল']}
    
    chars = []
    i = 0
    adjust = 0
    
    while(i < len(word)):
        if i+1 < len(word) and word[i+1] == '্':
            if word[i] == 'র':
                chars.append('র্')
                adjust = 0
                i+=2
            elif i+2 < len(word) and word[i+2] == 'য':
                chars.append(word[i-adjust:i+1])
                chars.append('্য')
                adjust = 0
                i+=3
            elif i+2 < len(word) and word[i+2] == 'র':
                # Treat '্র' as a seperate grapheme
                chars.append(word[i-adjust:i+1])
                chars.append('্র')
                # Keep '্র' icluded in the cluster
                # chars.append(word[i-adjust:i+3])
                if i+3 < len(word) and word[i+3] == '্' and i+4 < len(word) and word[i+4] == 'য':    
                    chars.append('্য')
                    i+=5
                else:
                    i+=3
                adjust = 0
            elif i+2 < len(word) and adjust!=0 and word[i-adjust:i+1] in forms_tripple_cluster \
                and word[i+2] in forms_tripple_cluster[word[i-adjust:i+1]]:
                if i+3 < len(word) and word[i+3] == '্':
                    adjust += 2
                    i+=2
                else:
                    chars.append(word[i-adjust:i+3])
                    adjust = 0
                    i+=3
            elif i+2 < len(word) and adjust==0 and word[i] in forms_cluster and word[i+2] in forms_cluster[word[i]]:
                if i+3 < len(word) and word[i+3] == '্':
                    adjust += 2
                    i+=2
                else:
                    chars.append(word[i-adjust:i+3])
                    adjust = 0
                    i+=3
            else:
                chars.append(word[i-adjust:i+1])
                chars.append('্')
                adjust = 0
                i+=2

        else:
            chars.append(word[i:i+1])
            i+=1

    
    #print(word)
    #print(chars)

    return chars

#Break every character
# def extract_graphemes(word):
    
#     return list(word)


In [8]:
chardict = {}

for word in ben_wordlist:
    word = normalize_word(word)
    for c in word:
        try:
            chardict[c] += 1
        except KeyError:
            chardict[c] = 1

print(len(chardict))
charlist = sorted(chardict.items(), key=lambda x:x[1], reverse=True)
chardict = dict(charlist)
print(chardict)
print()
print(sorted(chardict))

81
{'া': 367069, 'র': 307021, '্': 289632, 'ি': 220039, 'ে': 212704, 'ন': 172340, 'ক': 151716, 'স': 130915, 'ল': 119319, 'ব': 113135, 'ত': 110098, 'ম': 107774, 'ু': 85465, 'প': 85448, 'ট': 72365, 'দ': 67597, 'ো': 67263, 'য়': 64387, 'গ': 61244, 'য': 55211, 'জ': 50017, 'ী': 46803, 'ই': 44479, 'হ': 44278, 'শ': 41921, 'ড': 32821, 'ভ': 32142, 'চ': 27907, 'ও': 26371, 'ষ': 21270, 'ফ': 20943, 'ণ': 20758, 'অ': 20202, 'আ': 19818, 'ধ': 18847, 'উ': 18686, 'থ': 16164, 'এ': 16081, 'ং': 15604, 'ূ': 13973, 'খ': 13140, 'ড়': 9974, 'ঙ': 8707, 'ৃ': 8334, 'ছ': 7658, 'ঞ': 6168, 'ঁ': 5598, 'ঠ': 4782, 'ঘ': 4761, 'ৌ': 4060, '১': 3776, 'ৈ': 3388, 'ঃ': 3229, 'ৎ': 2790, '০': 2673, '২': 2222, 'ঝ': 1832, '৩': 1635, '৫': 1615, '৯': 1548, '৪': 1539, '৮': 1483, '৭': 1409, '৬': 1403, '।': 1042, 'ৰ': 910, 'ঈ': 549, 'ঢ': 475, 'ঐ': 330, 'ঊ': 303, 'ঋ': 290, 'ঢ়': 243, 'ঔ': 199, 'ৱ': 77, '\u09e4': 42, 'ঌ': 16, '৳': 6, 'ৄ': 5, 'ঽ': 3, '৺': 2, 'ৠ': 1}

['।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', 'এ', 'ঐ', 'ও',

In [9]:
grapheme_dict = {}

for word in ben_wordlist:
    graphemes = extract_graphemes(normalize_word(word))
    for grapheme in graphemes:
        try:
            grapheme_dict[grapheme] += 1
        except KeyError:
            grapheme_dict[grapheme] = 1

print(len(grapheme_dict))
grapheme_list = sorted(grapheme_dict.items(), key=lambda x:x[1], reverse=True)
grapheme_dict = dict(grapheme_list)
print(grapheme_dict)
print()
print(sorted(grapheme_dict))

228
{'া': 367069, 'ি': 220039, 'র': 215457, 'ে': 212704, 'ন': 133428, 'ক': 126500, 'ল': 103901, 'স': 97243, 'ব': 96630, 'ম': 95900, 'ু': 85465, 'ত': 79715, 'প': 74756, 'ো': 67263, 'য়': 64387, 'গ': 54218, '্র': 54177, 'ট': 51382, 'দ': 51061, 'ী': 46803, 'ই': 44479, 'জ': 43669, '্য': 43623, 'হ': 43459, 'শ': 38441, 'র্': 37387, 'ভ': 31276, 'ও': 26371, 'ড': 23798, 'চ': 22819, 'অ': 20202, 'আ': 19818, 'ফ': 19596, 'উ': 18686, 'ণ': 17547, 'এ': 16081, 'ং': 15604, 'ূ': 13973, 'ধ': 13560, 'খ': 12876, 'য': 11588, 'থ': 11024, 'ড়': 9974, 'ৃ': 8334, 'স্ট': 7874, 'ন্দ': 7548, 'ষ': 7226, 'ক্ষ': 6786, 'ন্ত': 6705, 'ছ': 6411, 'ঁ': 5598, 'স্ত': 5512, 'ন্ড': 5498, 'ঙ্গ': 5483, 'ন্ট': 5303, 'ঘ': 4606, 'ক্ত': 4476, 'ৌ': 4060, '্': 4039, 'স্থ': 3831, '১': 3776, 'ষ্ট': 3691, 'স্ক': 3533, 'ত্ত': 3517, 'ৈ': 3388, 'ক্স': 3342, 'ঃ': 3229, 'ন্স': 3007, 'ম্প': 2871, 'দ্ধ': 2841, 'ঠ': 2836, 'ৎ': 2790, 'স্ব': 2772, '০': 2673, 'ঞ্জ': 2453, 'ম্ব': 2362, 'ল্ল': 2325, '২': 2222, 'শ্ব': 2179, 'ন্ন': 2116, 'ণ্ড': 2000, 'জ্ঞ

In [10]:
wordlist_all = []

"""
[\u0980-\u09FF] is Bengali block
\u0964 is Devanagari Danda (Bangla dari uses this unicode)
[\u0020-\u002F] , [\u003A-\u003F] , [\u005B-\u0060], [\u007B-\u007E] are latin basic punctuations
[\u2018-\u2019] , [\u201C-\u201D] single and double quotations from the punctuation block
"""
pattern = re.compile(u'([\u0980-\u09FF]|\u0964|[\u0020-\u002F]|[\u003A-\u003F]|[\u005B-\u0060]|[\u007B-\u007E]|[\u2018-\u2019]|[\u201C-\u201D])+')
quotations = re.compile(u'([\u2018-\u2019]|[\u201C-\u201D])+')

for word in wordlist:
    if pattern.fullmatch(word) != None:
        if quotations.search(word) != None:
            if '\u2018' in word:
                word = word.replace('\u2018', '\u0027')
            if '\u2019' in word:
                word = word.replace('\u2019', '\u0027')
            if '\u201C' in word:
                word = word.replace('\u201C', '\u0022')
            if '\u201D' in word:
                word = word.replace('\u201D', '\u0022')
        
        wordlist_all.append(word)
    
print(len(wordlist_all))

/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel/__main__.py:9: FutureWarning: Possible nested set at position 22


695569


In [11]:
wordlist_all = list(set(wordlist_all))
print(len(wordlist_all))

691664


In [25]:
### save the list to a file
# with open(os.path.join(DATA_DIR, "dakshina/synthetic_wordlist_train.txt"), "w") as f:
with open(os.path.join(DATA_DIR, "dakshina/synthetic_wordlist_valid.txt"), "w") as f:
    for word in wordlist_all:
        f.write(word +"\n")

In [43]:
# with open(os.path.join(DATA_DIR, "one_biglist.txt"), 'r') as wordfile:
#     wordlist_all = [line.rstrip() for line in wordfile]

In [12]:
chardict = {}

for word in wordlist_all:
    word = normalize_word(word)
    for c in word:
        try:
            chardict[c] += 1
        except KeyError:
            chardict[c] = 1

print(len(chardict))
charlist = sorted(chardict.items(), key=lambda x:x[1], reverse=True)
chardict = dict(charlist)
print(chardict)
print()
print(sorted(chardict))

113
{'া': 582613, 'র': 462317, '্': 457196, 'ি': 351703, 'ে': 303893, 'ন': 276117, 'ক': 232820, 'স': 209459, 'ল': 189100, 'ব': 183280, 'ম': 173571, 'ত': 172131, 'প': 137515, 'ু': 129423, 'ট': 110539, 'দ': 105658, 'ো': 101782, 'য়': 99680, 'গ': 91901, 'য': 87547, 'জ': 80356, ',': 78524, 'ী': 73700, 'ই': 70483, 'হ': 69743, 'শ': 67958, '-': 62467, '।': 53736, 'ড': 52409, 'ভ': 49406, 'চ': 45621, 'এ': 39472, 'আ': 37327, 'ও': 36528, 'ষ': 33935, 'ফ': 33474, 'অ': 33048, '"': 32346, 'ণ': 31512, 'উ': 31179, 'ধ': 30118, "'": 29985, ')': 28811, '(': 27978, 'ং': 26137, 'থ': 25284, 'খ': 21861, 'ূ': 20501, '১': 17981, 'ড়': 15966, 'ছ': 15768, 'ঙ': 13716, '০': 13522, 'ৃ': 13198, 'ঞ': 10051, '২': 9926, 'ঁ': 9082, '৯': 8378, 'ঠ': 7855, 'ঘ': 7752, '.': 7379, '৫': 7151, '৩': 7010, '৮': 6544, 'ৌ': 6533, '৪': 6472, ':': 6110, '৬': 6085, '৭': 5963, 'ৈ': 5259, ';': 4771, 'ঃ': 4417, 'ৎ': 4368, '/': 3586, 'ঝ': 2945, 'ৰ': 1086, 'ঢ': 950, 'ঈ': 924, '?': 775, '!': 706, '%': 681, 'ঐ': 555, '[': 554, ']': 533, '|': 47

In [14]:
df = pd.DataFrame.from_dict(chardict, orient='index', columns=['Frequency'])
df.index.name = 'Character'
df.to_csv("character_frequency_synthetic2m.csv")

In [13]:
grapheme_dict = {}

for word in wordlist_all:
    graphemes = extract_graphemes(normalize_word(word))
    for grapheme in graphemes:
        try:
            grapheme_dict[grapheme] += 1
        except KeyError:
            grapheme_dict[grapheme] = 1

print(len(grapheme_dict))
grapheme_list = sorted(grapheme_dict.items(), key=lambda x:x[1], reverse=True)
grapheme_dict = dict(grapheme_list)
print(grapheme_dict)
print()
print(sorted(grapheme_dict))

260
{'া': 582613, 'ি': 351703, 'র': 317189, 'ে': 303893, 'ন': 215480, 'ক': 193055, 'ল': 164493, 'ব': 156858, 'স': 156722, 'ম': 154523, 'ু': 129423, 'ত': 124567, 'প': 120638, 'ো': 101782, 'য়': 99680, '্র': 85738, 'গ': 80498, 'দ': 80108, ',': 78524, 'ট': 77642, 'ী': 73700, 'ই': 70483, 'জ': 70111, 'হ': 68447, '্য': 68231, '-': 62467, 'শ': 62347, 'র্': 59390, '।': 53736, 'ভ': 48017, 'এ': 39472, 'ড': 38603, 'আ': 37327, 'চ': 37025, 'ও': 36528, 'অ': 33048, '"': 32346, 'ফ': 31474, 'উ': 31179, "'": 29985, ')': 28811, '(': 27978, 'ণ': 26663, 'ং': 26137, 'ধ': 21828, 'খ': 21466, 'ূ': 20501, 'য': 19316, '১': 17981, 'থ': 17648, 'ড়': 15966, 'ছ': 13533, '০': 13522, 'ৃ': 13198, 'স্ট': 12261, 'ষ': 11704, 'ন্দ': 11641, 'ক্ষ': 10811, 'ন্ত': 10433, '২': 9926, 'ঁ': 9082, 'ঙ্গ': 8899, 'ন্ড': 8625, 'স্ত': 8619, '৯': 8378, 'ন্ট': 8227, 'ঘ': 7532, '.': 7379, '৫': 7151, 'ক্ত': 7096, '৩': 7010, '্': 6871, '৮': 6544, 'ৌ': 6533, '৪': 6472, ':': 6110, '৬': 6085, '৭': 5963, 'ষ্ট': 5728, 'স্থ': 5703, 'ত্ত': 5638, 'স্ক

In [16]:
df = pd.DataFrame.from_dict(grapheme_dict, orient='index', columns=['Frequency'])
df.index.name = 'Grapheme'
df.to_csv("grapheme_frequency_synthetic2m.csv")

In [18]:
with open(os.path.join(DATA_DIR, "dakshina/synthetic_wordlist_valid.txt"), 'r') as wordfile:
    wordlist_valid = [line.rstrip() for line in wordfile]

grapheme_dict_valid = {}

for word in wordlist_valid:
    graphemes = extract_graphemes(normalize_word(word))
    for grapheme in graphemes:
        try:
            grapheme_dict_valid[grapheme] += 1
        except KeyError:
            grapheme_dict_valid[grapheme] = 1

print(len(grapheme_dict_valid))
grapheme_list_valid = sorted(grapheme_dict_valid.items(), key=lambda x:x[1], reverse=True)
grapheme_dict_valid = dict(grapheme_list_valid)
print(grapheme_dict_valid)
print()
print(sorted(grapheme_dict_valid))

230
{'া': 43215, 'ি': 25694, 'র': 24874, 'ে': 24254, 'ন': 16044, 'ক': 14164, 'ব': 12317, 'ল': 11918, 'স': 11307, 'ম': 11171, 'ত': 10442, 'প': 9641, 'ু': 9161, 'য়': 7285, 'ো': 6758, '্র': 6742, 'দ': 6420, 'গ': 5879, 'ী': 5648, 'ট': 5479, ',': 5414, 'জ': 5256, '্য': 5101, 'শ': 5076, 'হ': 4990, 'র্': 4667, 'ই': 4618, 'ভ': 3535, '-': 2861, 'চ': 2762, 'অ': 2725, 'আ': 2713, 'এ': 2399, 'ও': 2351, 'ড': 2325, 'উ': 2313, 'ণ': 2163, 'ং': 2117, 'ফ': 2058, 'ধ': 1903, 'খ': 1768, 'ূ': 1564, 'য': 1522, '১': 1500, '(': 1314, 'থ': 1303, ')': 1300, 'ড়': 1299, '"': 1234, 'ছ': 1124, "'": 1066, 'ৃ': 1008, 'ক্ষ': 1005, '০': 992, 'ষ': 988, 'ন্দ': 820, 'স্ট': 817, 'ন্ত': 808, '৯': 778, '২': 740, 'ঁ': 699, 'স্ত': 672, 'ঙ্গ': 646, 'ঘ': 630, 'ন্ড': 620, '৮': 539, 'ৌ': 530, 'স্থ': 528, 'ক্ত': 522, 'ন্ট': 519, '৫': 515, '৩': 476, 'ষ্ট': 451, '৪': 449, '৭': 444, '৬': 442, 'ৈ': 441, 'ঠ': 411, 'ত্ত': 404, 'দ্ধ': 395, 'ম্প': 394, 'স্ক': 393, 'স্ব': 374, 'ক্স': 361, 'ন্স': 343, ':': 340, '্': 323, 'ৎ': 319, 'ঞ্জ': 289, 

In [24]:
diff = set(sorted(grapheme_dict)) - set(sorted(grapheme_dict_valid))
print(diff)

{'ধ্ন', 'জ্ঝ', 'ৡ', 'ণ্ম', 'ন্ত্ব', 'স্ত্ব', '<', 'ম্প্ল', 'ঙ্ম', 'ব্ভ', 'দ্দ্ব', 'দ্ধ্ব', '&', 'ল্স', 'ণ্ঢ', 'ঌ', 'ৠ', 'হ্ল', 'ঞ্ঝ', 'ৄ', 'ঽ', '~', '}', '_', '\u09e4', '৳', 'ক্ন', 'ষ্ব', '\\', 'ট্ব', '{'}


In [25]:
diff = set(sorted(grapheme_dict_valid)) - set(sorted(grapheme_dict))
print(diff)

set()


In [20]:
with open('/home/ec2-user/word_level_ocr/pritom/pipeline/ocr_pipelines/ckpts/vgg342/inv_grapheme_dict_i_12lakh_Synth.pickle', 'rb') as handle:
    inv_grapheme_dict = pickle.load(handle)

inv_grapheme_dict = {v: k for k, v in inv_grapheme_dict.items()}

In [21]:
diff = set(sorted(inv_grapheme_dict)) - set(sorted(grapheme_dict))
print(diff)

{'Ÿ', 'ÿ', '\uf061', '‰', 'œ', '¹', '\uf0be', '\uf046', '\uf071', 'ো', '\u200f', '§', '\u09e5', '\uf020', '╛', '\u200c', '\uf028', '╟', '¤', 'ð', '\uf06c', '\uf03a', 'Б', '\uf06e', '\uf0fa', '╕', '¸', '\uf06f', '\u200d', '╨', '╚', '\uf029', '‚', '°', '‘', '\uf0d7', '॥', 'ৌ', 'ƒ', '\uf02e', 'Í', '╓', 'Ê', '\uf0ce', '“', 'ú', 'Ö', 'è', '«', '\uf041', '৵', '║', 'Ü', '\uf0ab', '\uf03d', '±', '¦', '\uf031', '\uf0b4', '\uf04f', '®', 'ª', 'È', '¡', '\uf02b', '∏', '�', 'Å', '\x8f', '╠', '\uf033', '╞', '²', 'Ø', '\x9d', '\uf070', '–', '\uf02d', 'Ä', 'æ', '\uf0ae', '\uf05c', 'ø', '©', '\uf044', '\xad', '।', '¯', 'ä', '\uf04e', '\uf03e', '´', '÷', '•', '×', '৴', '\u200b', '¬', '”', 'ý', '\uf048', '\uf062', 'β', '—', '\uf06d', '╬', '›', '<eow>', '¥', 'š', '’', '┤'}


In [3]:
# initializing string
test_str = '¸.‚,।৷'
  
# printing original String
print("The original string is : " + str(test_str))
  
# using sub() to perform substitutions
# ord() for conversion.
res = (re.sub('.', lambda x: r'\u % 04X ' % ord(x.group()), test_str))
  
# printing result 
print("The unicode converted String : " + str(res))

The original string is : ¸.‚,।৷
The unicode converted String : \u  0B8 \u  02E \u  201A \u  02C \u  964 \u  9F7 


In [13]:
# Create the workbook and sheet for Excel
workbook = openpyxl.Workbook()
sheet = workbook.active

# openpyxl does things based on 1 instead of 0
row = 1
for key, value in grapheme_dict.items():
    # Put the key in the first column for each key in the dictionary
    try:
        sheet.cell(row=row, column=1, value=key)
        sheet.cell(row=row, column=2, value=value)
        row += 1
    except:
        continue

workbook.save(filename="grapheme_frequency.xlsx")

### Label mapping 

In [6]:
files = sorted(os.listdir(os.path.join(DATA_DIR, "synthetic_ocr/train")), key=lambda x: int(x.split('.')[0]))

In [7]:
print(len(files))

1929054


In [46]:
print(files[0:20])

['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg']


In [47]:
with open(os.path.join(DATA_DIR, "synthetic_ocr/train_labels.txt"), 'r') as wordfile:
    mappings = [line.rstrip() for line in wordfile]
    
labels_dict = {filename: label for filename, label in (mapping.split() for mapping in mappings)}

In [48]:
len(labels_dict)

2572072

In [49]:
labels_dict['10.jpg']

"'শাহানা-ই-মান্ডি'"

In [50]:
filenames = sorted(labels_dict.keys(), key=lambda x: int(x.split('.')[0]))

In [51]:
print(filenames[:10])

['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg']


In [52]:
for i in range(10):
    print(labels_dict[filenames[i]])

গোত্রপ্রধান,
গেমকে
স্ক্লেরোসিস,
সমলোচনার
সেরিব্রাল
আশাবাদ
স্কুফ্টান
নিন্মভাগ
(খাঁটি
'শাহানা-ই-মান্ডি'


### Synthetic Test Set

In [78]:
with open(os.path.join(DATA_DIR, "dakshina/wiki-filt.valid.text.shuf.txt"), 'r') as wordfile:
    lines = [line.rstrip() for line in wordfile]
    print("Total lines: {}".format(len(lines)))

Total lines: 26720


In [79]:
wordlist_valid = []

for line in lines:
    words = line.split()
    for word in words:
        wordlist_valid.append(word)
        
print(len(wordlist_valid))
wordlist_valid = list(set(wordlist_valid))
print(len(wordlist_valid))

325713
61978


In [80]:
wordlist_valid_all = []

"""
[\u0980-\u09FF] is Bengali block
\u0964 is Devanagari Danda (Bangla dari uses this unicode)
[\u0020-\u002F] , [\u003A-\u003F] , [\u005B-\u0060], [\u007B-\u007E] are latin basic punctuations
[\u2018-\u2019] , [\u201C-\u201D] single and double quotations from the punctuation block
"""
pattern = re.compile(u'([\u0980-\u09FF]|\u0964|[\u0020-\u002F]|[\u003A-\u003F]|[\u005B-\u0060]|[\u007B-\u007E]|[\u2018-\u2019]|[\u201C-\u201D])+')
quotations = re.compile(u'([\u2018-\u2019]|[\u201C-\u201D])+')

for word in wordlist_valid:
    if pattern.fullmatch(word) != None:
        if quotations.search(word) != None:
            if '\u2018' in word:
                word = word.replace('\u2018', '\u0027')
            if '\u2019' in word:
                word = word.replace('\u2019', '\u0027')
            if '\u201C' in word:
                word = word.replace('\u201C', '\u0022')
            if '\u201D' in word:
                word = word.replace('\u201D', '\u0022')
        
        wordlist_valid_all.append(word)
    
print(len(wordlist_valid_all))

61235


In [81]:
with open(os.path.join(DATA_DIR, "150000.txt"), 'r') as wordfile:
    lines = [line.rstrip() for line in wordfile]
    print("Total lines: {}".format(len(lines)))

Total lines: 149998


In [82]:
wordlist_test = []

for line in lines:
    words = line.split()
    for word in words:
        wordlist_test.append(word)
        
print(len(wordlist_test))
wordlist_test = list(set(wordlist_test))
print(len(wordlist_test))

149998
149998


In [83]:
wordlist_test_all = []

"""
[\u0980-\u09FF] is Bengali block
\u0964 is Devanagari Danda (Bangla dari uses this unicode)
[\u0020-\u002F] , [\u003A-\u003F] , [\u005B-\u0060], [\u007B-\u007E] are latin basic punctuations
[\u2018-\u2019] , [\u201C-\u201D] single and double quotations from the punctuation block
"""
pattern = re.compile(u'([\u0980-\u09FF]|\u0964|[\u0020-\u002F]|[\u003A-\u003F]|[\u005B-\u0060]|[\u007B-\u007E]|[\u2018-\u2019]|[\u201C-\u201D])+')
quotations = re.compile(u'([\u2018-\u2019]|[\u201C-\u201D])+')

for word in wordlist_test:
    if pattern.fullmatch(word) != None:
        if quotations.search(word) != None:
            if '\u2018' in word:
                word = word.replace('\u2018', '\u0027')
            if '\u2019' in word:
                word = word.replace('\u2019', '\u0027')
            if '\u201C' in word:
                word = word.replace('\u201C', '\u0022')
            if '\u201D' in word:
                word = word.replace('\u201D', '\u0022')
        
        wordlist_test_all.append(word)
    
print(len(wordlist_test_all))

145824


In [85]:
wordlist_test = list(set(wordlist_test_all) - (set(wordlist_all) | set(wordlist_valid_all)))
print(len(wordlist_test))

75138


In [87]:
### save the list to a file
with open(os.path.join(DATA_DIR, "dakshina/synthetic_wordlist_test.txt"), "w") as f:
    for word in wordlist_test:
        f.write(word +"\n")